## What可视化

2D Grand CAM可视化模块

In [ ]:
## 获得视频教程
from onekey_algo.custom.Manager import onekey_show
onekey_show('What模型可视化')

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt

mydir = r'J:\4type\resnet18_4517\viz\picture\OP'
samples = [os.path.join(mydir, f) for f in os.listdir(mydir) if f.endswith('.jpg') or f.endswith('.png')]
samples

## 确定可视化模型

通过关键词获取要提取那一层进行可视化。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| AlexNet      | alexnet                                                      |
| VGG          | vgg11, vgg11_bn, vgg13, vgg13_bn, vgg16, vgg16_bn, vgg19_bn, vgg19 |
| ResNet       | resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2 |
| DenseNet     | densenet121, densenet169, densenet201, densenet161           |
| Inception    | googlenet, inception_v3                                      |
| SqueezeNet   | squeezenet1_0, squeezenet1_1                                 |
| ShuffleNetV2 | shufflenet_v2_x2_0, shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5 |
| MobileNet    | mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small         |
| MNASNet      | mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3              |

In [ ]:
from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey

model_name = 'resnet18'
# model, transformer, device = init_from_model(model_name=model_name, transfer_learning=False)
model, transformer, device = init_from_onekey(r'J:\4type\resnet18_4517\viz')
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

## 可视化卷积层

`Feature name:` 之后的名称为要可视化的层，例如`layer4.2.conv3`, 一般深度学习特征提取最后一层卷积层

** 注意 ** : 可视化的层，一定为带有`conv`的卷积层，而且一般是最后一层。

In [ ]:
target_layer = "layer4.1.conv2"
gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)

## 打印可视化界面

In [ ]:
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch

for sample in samples:
    img = default_loader(sample)
    sample_ = transformer(img)
    sample_  = sample_.view(1, *sample_.size()).to(device)
    res_cam = gradcam(x=sample_, class_idx=None)
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
    axes[0].imshow(res_cam[0][0].cpu(), cmap='jet')
    axes[1].imshow(img.resize(sample_.size()[2:]))
    plt.show()
    
    plt.figure(figsize=(10, 10))
    plt.imshow(show_cam_on_image(img.resize(sample_.size()[2:]), res_cam[0][0].cpu(), use_rgb=True))